# Prepare the Habitat data


### Import libraries and set up environment

In [27]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [28]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# Make sure "Carib_mol_30km" is in the project (made in EC_study_area_prep)

In [ ]:
# Add the GMW (2020) data to the ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"O:\f00_data\GMW_001_GlobalMangroveWatch\01_GMW_001_GlobalMangroveWatch\01_Data\gmw_v3\gmw_v3_2020_vec.shp",
    Output_Geodatabase= output_gdb
)


In [ ]:
# Add the Global Distribution of Coral Reefs (v4.1) (poly and point) data to the ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"O:\f00_data\WCMC-008-GlobalDistributionOfCoralReefs2010\14_001_WCMC008_CoralReefs2021_v4_1\01_Data\WCMC008_CoralReef2021_Py_v4_1.shp;O:\f00_data\WCMC-008-GlobalDistributionOfCoralReefs2010\14_001_WCMC008_CoralReefs2021_v4_1\01_Data\WCMC008_CoralReef2021_Pt_v4_1.shp",
    Output_Geodatabase= output_gdb
)

In [ ]:
# Add the Global Distribution of Saltmarshes (v6.1) (poly only) data to the ArcGIS project
# Note that points aren't used in this analysis, based on the recommendation in the metadata

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"O:\f00_data\WCMC-027-GlobalDistributionOfSaltmarsh\WCMC027_Saltmarsh_v6_1\01_Data\WCMC027_Saltmarshes_Py_v6_1.shp",
    Output_Geodatabase= output_gdb
)


In [ ]:
# Add the Global Distribution of Seagrasses (v7.1) (poly and point) data to the ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"O:\f00_data\WCMC-013-014-GlobalDistributionOfSeagrasses2005\014_001_WCMC013-014_SeagrassPtPy2021_v7_1\01_Data\WCMC013_014_Seagrasses_Py_v7_1.shp;O:\f00_data\WCMC-013-014-GlobalDistributionOfSeagrasses2005\014_001_WCMC013-014_SeagrassPtPy2021_v7_1\01_Data\WCMC013_014_Seagrasses_Pt_v7_1.shp",
    Output_Geodatabase= output_gdb
)

In [ ]:
# Add the Global Distribution of Seamounts and Knolls data to the ArcGIS project
# Note that only the base area shapefiles are used

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"O:\f00_data\ZSL-002-ModelledSeamounts2011\DownloadPack-14_001_ZSL002_ModelledSeamounts2011_v1\01_Data\KnollsBaseArea\KnollsBaseArea.shp;O:\f00_data\ZSL-002-ModelledSeamounts2011\DownloadPack-14_001_ZSL002_ModelledSeamounts2011_v1\01_Data\SeamountsBaseArea\SeamountsBaseArea.shp",
    Output_Geodatabase= output_gdb
)

In [ ]:
# Buffer the coral reef point data to 1 km area

points = "WCMC008_CoralReef2021_Pt_v4_1"
out_feature = "\\Coral_pt_mol_buf"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def buffer_points(points):
    #Project to Mollweide
    arcpy.management.Project(points, "Coral_pt_mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    
    #Calculate radius of buffer on WDPA_pt_Mol layer
    arcpy.management.CalculateField("Coral_pt_mol", "radius", "math.sqrt(1/math.pi)*1000", "PYTHON3")

    #buffer to radius.
    #points without area were automatically removed
    arcpy.analysis.Buffer("Coral_pt_mol", output_gdb + out_feature, "radius", "FULL", "ROUND", "NONE", None, "PLANAR")

    #checked area to make sure it matched
    arcpy.management.CalculateGeometryAttributes(out_feature, "area AREA", '', "SQUARE_KILOMETERS", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")

buffer_points(points)


In [ ]:
# Append the buffered coral reef points to the coral reef polygons

buffered_points = "Coral_pt_mol_buf"
poly = "WCMC008_CoralReef2021_Py_v4_1"
out_feature = "\\Coral_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"


def append_points_to_poly():
    #Project to Mollweide
    arcpy.management.Project(poly, output_gdb + "\\Coral_poly_Mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

    #repair geometry
    arcpy.management.RepairGeometry("Coral_poly_Mol", "DELETE_NULL", "OGC")
    
    #rename it to get ready to append it with points
    arcpy.conversion.ExportFeatures("Coral_poly_Mol", output_gdb + out_feature)
    
    #Append the Coral_mol with the point layer 
    #Note that append worked better than merge because merge didn't keep the attributes of the points
    arcpy.management.Append(buffered_points, output_gdb + out_feature, "NO_TEST")
    
    #repair geometry
    arcpy.management.RepairGeometry(out_feature, "DELETE_NULL", "OGC")

append_points_to_poly()

In [ ]:
# Buffer the seagrass point data to 1 km area

points = "WCMC013_014_Seagrasses_Pt_v7_1"
out_feature = "\\Seagrass_pt_mol_buf"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def buffer_points(points):
    #Project to Mollweide
    arcpy.management.Project(points, "Seagrass_pt_mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    
    #Calculate radius of buffer on WDPA_pt_Mol layer
    arcpy.management.CalculateField("Seagrass_pt_mol", "radius", "math.sqrt(1/math.pi)*1000", "PYTHON3")

    #buffer to radius.
    #points without area were automatically removed
    arcpy.analysis.Buffer("Seagrass_pt_mol", output_gdb + out_feature, "radius", "FULL", "ROUND", "NONE", None, "PLANAR")

    #checked area to make sure it matched
    arcpy.management.CalculateGeometryAttributes(out_feature, "area AREA", '', "SQUARE_KILOMETERS", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")

buffer_points(points)

In [ ]:
# Append the buffered seagrass points to the coral reef polygons

buffered_points = "Seagrass_pt_mol_buf"
poly = "WCMC013_014_Seagrasses_Py_v7_1"
out_feature = "\\Seagrass_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"


def append_points_to_poly():
    #Project to Mollweide
    arcpy.management.Project(poly, output_gdb + "\\Seagrass_poly_Mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

    #repair geometry
    arcpy.management.RepairGeometry("Seagrass_poly_Mol", "DELETE_NULL", "OGC")
    
    #rename it to get ready to append it with points
    arcpy.conversion.ExportFeatures("Seagrass_poly_Mol", output_gdb + out_feature)
    
    #Append the Coral_mol with the point layer 
    #Note that append worked better than merge because merge didn't keep the attributes of the points
    arcpy.management.Append(buffered_points, output_gdb + out_feature, "NO_TEST")
    
    #repair geometry
    arcpy.management.RepairGeometry(out_feature, "DELETE_NULL", "OGC")

append_points_to_poly()

In [ ]:
# Reproject the other habitat layers (mangroves, saltmarsh, seamounts, and knolls) to Mollweide

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

mangroves = "gmw_v3_2020_vec"
saltmarsh = "WCMC027_Saltmarshes_Py_v6_1"
seamounts = "SeamountsBaseArea"
knolls = "KnollsBaseArea"

habitats_4 = {mangroves : "\\mangroves_mol", saltmarsh : "\\saltmarsh_mol", seamounts : "\\seamounts_mol", knolls : "\\knolls_mol"}

def mol():
    for habitat, output_name in habitats_4.items():
        #Project all four datasets to Mollweide
        arcpy.management.Project(habitat, output_gdb + output_name, 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

mol()

In [4]:
# Clip all habitat layers to the 30km buffered study area and dissolve

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

clip_feature = "Carib_mol_30km"

# Inputs
mangroves = "mangroves_mol"
saltmarsh = "saltmarsh_mol"
seamounts = "seamounts_mol"
knolls = "knolls_mol"
coral = "Coral_mol"
seagrass = "Seagrass_mol"

# Dictionary of habitats
habitats_6 = {mangroves : "\\mangroves_mol_Carib30_dis", saltmarsh : "\\saltmarsh_mol_Carib30_dis", seamounts : "\\seamounts_mol_Carib30_dis", knolls : "\\knolls_mol_Carib30_dis", coral : "\\coral_mol_Carib30_dis", seagrass : "\\seagrass_mol_Carib30_dis"}

def habitat_clip_dis(habitat, output_name):        
    #repair geometry
    arcpy.management.RepairGeometry(habitat, "DELETE_NULL", "OGC")
    
    # clip to the 30km Caribbean study area in memory
    clipped = arcpy.analysis.Clip(habitat, clip_feature, "in_memory/clipped")
        
    # dissolve to a flat layer in memory. Make them singlepart for faster processing
    dissolved = arcpy.analysis.PairwiseDissolve(
        in_features= clipped,
        out_feature_class= "in_memory/dissolved",
        dissolve_field=None,
        statistics_fields=None,
        multi_part="SINGLE_PART",
        concatenation_separator=""
    )
        
    # Copy the dissolved result to the output geodatabase
    arcpy.management.CopyFeatures(dissolved, output_gdb + output_name)
    
    #repair geometry
    arcpy.management.RepairGeometry(output_name, "DELETE_NULL", "OGC")

# loop through the habitat layers and process them
for habitat, output_name in habitats_6.items():
    habitat_clip_dis(habitat, output_name)

In [9]:
# Merge and dissolve seamounts and knolls into a single layer because they will be treated as a single habitat type

in_feature_1 = "seamounts_mol_Carib30_dis"
in_feature_2 = "knolls_mol_Carib30_dis"
out_feature = "\\seamount_knoll_mol_Carib30_dis"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def seamount_knoll():
    # merge seamounts and knolls
    merged = arcpy.management.Merge(
        inputs=[in_feature_1, in_feature_2],
        output= "in_memory/merged",
        field_mappings= None,
        add_source="NO_SOURCE_INFO"
    )
    
    # dissolve 
    dissolved = arcpy.analysis.PairwiseDissolve(
        in_features= merged,
        out_feature_class= "in_memory/dissolved",
        dissolve_field=None,
        statistics_fields=None,
        multi_part="SINGLE_PART",
        concatenation_separator=""
    )
    
    # Copy the dissolved result to the output geodatabase
    arcpy.management.CopyFeatures(dissolved, output_gdb + out_feature)
    
    # repair geometry
    arcpy.management.RepairGeometry(out_feature, "DELETE_NULL", "OGC")
    
seamount_knoll()

In [30]:
# Add a column indicating the habitat type for each habitat dataset

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

# Inputs
mangroves = "mangroves_mol_Carib30_dis"
saltmarsh = "saltmarsh_mol_Carib30_dis"
seamount_knoll = "seamount_knoll_mol_Carib30_dis"
coral = "coral_mol_Carib30_dis"
seagrass = "seagrass_mol_Carib30_dis"

# Dictionary of habitats
habitats_5 = {mangroves : "mangrove", saltmarsh : "saltmarsh", seamount_knoll : "seamount_knoll", coral : "coral", seagrass : "seagrass"}

def habitat_field(habitat, habitat_name):
    #add field to indicate habitat type
    arcpy.management.CalculateField(
        in_table=habitat,
        field="Habitat",
        expression= f"'{habitat_name}'",
        expression_type="PYTHON3",
        code_block="",
        field_type="TEXT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

# loop through the habitat layers and process them
for habitat, habitat_name in habitats_5.items():
    habitat_field(habitat, habitat_name)
